In [5]:
from oyv.imports import *
import pandas as pd
import glob
from oyv.map import quickmap
from cartopy import feature as cfeature
from cartopy import crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

In [6]:
figdir = '/home/oyvindl/work/projects/barents_glaciers/figures/runoff_mod/'

In [7]:
ddir = '/home/oyvindl/work/projects/barents_glaciers/data/runoff_model/most_recent_louise/Avrenning_Austfonna_2022/'
fns = glob.glob(ddir + '*.dat')

In [8]:
fact_m3_day =1e9/86400 # Double check - assuming 1000 kg/m3 

In [9]:
A = pd.read_csv(fns[0], sep = ',')
ARR = A.to_numpy()

# Define indices

In [72]:
nns_ss = [513, 518, 522, 524, 525, 526] # 
nns_hb = [521, 520] # 

In [25]:
B = Bunch()
B.lon, B.lat = A.Longitude.to_numpy(), A.Latitude .to_numpy()

for key in ['t', 'runoff_all']:
    B[key] = np.ma.array([])

B.sum_FWC_2018_2020 = np.zeros(ARR.shape[0])

for yr in np.arange(2014, 2023):
    fn_yr = 'runoff_Austfonna_%i_k_0.9.dat'%yr
    A = pd.read_csv(ddir + fn_yr, sep = ',')
    ARR = A.to_numpy()
    
    ndays = ARR.shape[1]-4
    B.t = np.concatenate([B.t, num2date(date2num(dt.date(yr, 1, 1)) +
                                        np.arange(ndays))])
    B.runoff_all = np.concatenate([B.runoff_all, 
                                   ARR[:, 4:].sum(axis = 0)])
    if yr < 2021:
        B.sum_FWC_2018_2020 += ARR[:, 4:].sum(axis = 1)

### Map

In [10]:
# Define projections
cproj = ccrs.Orthographic(central_longitude=20, 
                          central_latitude=78, globe=None)
proj_33n = ccrs.TransverseMercator(central_longitude=15, central_latitude=0.0, scale_factor=0.9996,
              false_easting=500000.0, false_northing = 0)

/tmp/ipykernel_21963/209499429.py:4: UserWarning: The default value for the *approx* keyword argument to TransverseMercator will change from True to False after 0.18.
  proj_33n = ccrs.TransverseMercator(central_longitude=15, central_latitude=0.0, scale_factor=0.9996,


In [42]:
# Load glacier basin data shapefile
coast_name = '/home/oyvindl/work/projects/barents_glaciers/data/GI2020_jack/GI2020_merge_test.shp'
shape_feature = ShapelyFeature(Reader(coast_name).geometries(),
                               proj_33n,  facecolor='w', edgecolor = 'k', lw = 0.2)

In [51]:
ca()

In [52]:
fig = plt.figure()
ax = plt.axes(projection=cproj)
ax.coastlines()
ax.add_feature(cfeature.LAND, color = [0.5, 0.5, 0.5])
ax.set_extent([18, 28, 79, 80.8], crs=ccrs.PlateCarree())
ax.add_feature(shape_feature)
ax.set_facecolor(col.col('b', sec = True))

In [53]:
ax.scatter(A.Longitude.to_numpy(), A.Latitude.to_numpy(), 
           2, color = 'k', transform = ccrs.PlateCarree())


In [49]:
maxsize = 1500

In [29]:
ax.scatter(A.Longitude, A.Latitude, s = B.sum_FWC_2018_2020/B.sum_FWC_2018_2020.max()*maxsize/3, 
          marker = 'o', alpha = 0.7,
          edgecolor = col.col('r'), color = col.col('r', sec = True), 
           transform = ccrs.PlateCarree(), zorder = 10)


In [32]:
# Legend
legcoords = (30.2, 80.4)
legsize = 0.5
ax.scatter(*legcoords, s = legsize/B.sum_FWC_2018_2020.max()*maxsize, marker = 'o', 
          edgecolor = col.col('r'), color = col.col('r', sec = True), transform = ccrs.PlateCarree(), zorder = 10)
ax.text(legcoords[0], legcoords[1]+0.06, '%.0f MT yr$^{-1}$'%(legsize*1e3),  
        zorder = 10, va = 'bottom', ha = 'center', transform = ccrs.PlateCarree(),)

Text(30.2, 80.46000000000001, '500 MT yr$^{-1}$')